<a href="https://colab.research.google.com/github/Natural-Language-Processing-YU/Module-5-Assignment/blob/main/scripts/Part%20II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word2Vec Application Tutorial

**Note: This tutorial is provided by Google Research.**


In this tutorial, we go over basic operations on word vectors. There are many Natural Language Processing (NLP) libraries in Python, such as [NLTK](https://www.nltk.org/), [gensim](https://radimrehurek.com/gensim/), and [spaCy](https://spacy.io/). All of them have their own strength and focus. NLTK is one of the first comprehensive Python libraries for computational linguistics and has a big community. If you have worked on NLP, you probably have heard of it or used it. Gensim is a popular library for topic modeling. It also provides many functionalities similar to NLTK. It supports word embeddings and you can even train word embeddings using gensim. SpaCy is another popular NLP library and it provides built-in support for word vectors. We will use spaCy in this tutorial.  \\
<br>
You will learn:


1.   Popular Python machine learning packages (spaCy, sklearn)
2.   Calculating word similarity using Word2Vec model
3.   Word analogy analysis
4.   Calculating sentence similarity using Word2Vec model
5.   Dimension reduction techniques for high-dimensional vectors
6.   Visualizing Word2Vec in 2D space
7.   Sentiment analysis using logistic regression and Word2Vec



### Preliminary
First, let's install the spaCy Python library and download their model for the English language. We only need to do it once. Then we can import the spaCy library and other useful libraries such as numpy (used for linear algebra and vector operations in Python). We can load our downloaded English model in our environment.

In [1]:
# Only needs to be run once at the top of the notebook
!pip install spacy
!python -m spacy download en_core_web_lg

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for spacy from https://files.pythonhosted.org/packages/92/fb/d1f0605e1e8627226c6c96053fe1632e9a04a3fbcd8b5d715528cb95eb97/spacy-3.7.4-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for spacy-legacy<3.1.0,>=3.0.11 from https://files.pythonhosted.org/packages/c3/55/12e842c70ff8828e34e543a2c7176dac4da006ca6901c9e8b43efab8bc6b/spacy_legacy-3.0.12-py2.py3-none-any.whl.metadata
  Obtaining dependency information for spacy-loggers<2.0.0,>=1.0.0 from https://files.pythonhosted.org/packages/33/78/d1a1a026ef3af911159398c939b1509d5c36fe524c7b644f34a5146c4e16/spacy_loggers-1.0.5-py3-none-any.whl.metadata
  Obtaining dependency information for murmurhash<1.1.0,>=0.28.0 from https://files.pythonhosted.org/packages/71/46/af01a20ec368bd9cb49a1d2df15e3eca113bbf6952cc1f2a47f1c6801a7f/murmurhash-1.0.10-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/587.7 MB ? eta -:--:--
     ---------------------------------------- 0.2/587.7 MB 5.3 MB/s eta 0:01:52
     ---------------------------------------- 0.5/587.7 MB 6.6 MB/s eta 0:01:30
     ---------------------------------------- 0.9/587.7 MB 7.3 MB/s eta 0:01:21
     ---------------------------------------- 1.4/587.7 MB 8.8 MB/s eta 0:01:07
     ---------------------------------------- 1.8/587.7 MB 8.7 MB/s eta 0:01:08
     ---------------------------------------- 2.3/587.7 MB 9.6 MB/s eta 0:01:02
     ---------------------------------------- 2.8/587.7 MB 9.8 MB/s eta 0:01:00
     --------------------------------------- 3.3/587.7 MB 10.1 MB/s eta 0:00:59
     --------------------------------------- 3.9/587.7 MB 10.9 MB/s eta 0:00:54
     --------------------------------------- 4.5/587.7 MB 11.0 MB/s eta 0:00:54
     --------------------------------------- 5.0/

In [2]:
# import packages
import spacy
import numpy as np
import csv
from sklearn.manifold import TSNE
from sklearn import linear_model
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import clear_output

In [3]:
nlp = spacy.load('en_core_web_lg')  # load the English model

### Word Similarity
By representing words in vectors, we can use linear algebra and vector space models to analyze the relationship between words. One simple task is to calculate the cosine of two word vectors, namely the cosine similarity. This cosine similarity measures the semantic similarity of words. While the value ranges from -1 to 1, it is usually used in the non-negative space [0, 1] where 0 means 0 similarity and 1 means extremely similar or even identical.

In order to calculate the cosine similarity between words, we have to know their vector representations first, which are provided by the Word2Vec model. In the spaCy English model, these vector representations (pretrained using Word2Vec) are already provided. All we need to do is to retrieve these words from the spaCy English model and we will have access to these vector representations. \\
<br>
![cosine_sim](https://engineering.aweber.com/wp-content/uploads/2013/02/4AUbj.png)

In [4]:
# retrieve words from the English model vocabulary
cat = nlp.vocab['cat']
dog = nlp.vocab['dog']
car = nlp.vocab['car']

# print the dimension of word vectors
print('vector length:', len(cat.vector))

# print the word vector
print('cat:', cat.vector)

vector length: 300
cat: [ 3.7032e+00  4.1982e+00 -5.0002e+00 -1.1322e+01  3.1702e-02 -1.0255e+00
 -3.0870e+00 -3.7327e+00  5.3875e-01  3.5679e+00  6.9276e+00  1.5793e+00
  5.1188e-01  3.1868e+00  6.1534e+00 -4.8941e+00 -2.9959e-01 -3.6276e+00
  2.3825e+00 -1.4402e+00 -4.7577e+00  4.3607e+00 -4.9814e+00 -3.6672e+00
 -1.8052e+00 -2.1888e+00 -4.2875e+00  5.5712e+00 -5.2875e+00 -1.8346e+00
 -2.2015e+00 -7.7091e-01 -4.8260e+00  1.2464e+00 -1.7945e+00 -8.1280e+00
  1.9994e+00  1.1413e+00  3.8032e+00 -2.8783e+00 -4.2136e-01 -4.4177e+00
  7.7456e+00  4.9535e+00  1.7402e+00  1.8275e-01  2.4218e+00 -3.1496e+00
 -3.8057e-02 -2.9818e+00  8.3396e-01  1.1531e+01  3.5684e+00  2.5970e+00
 -2.8438e+00  3.2755e+00  4.5674e+00  3.2219e+00  3.4206e+00  1.1200e-01
  1.0303e-01 -5.8396e+00  4.6370e-01  2.7750e+00 -5.3713e+00 -5.0247e+00
 -2.0212e+00  5.8772e-01  1.1569e+00  1.3224e+00  4.3994e+00  2.0444e+00
  2.1343e+00 -1.9023e+00  2.1469e+00 -2.9085e+00  4.8429e-01 -3.3544e-01
  1.4484e+00 -1.5770e+00 -1

Try to retrieve some other words and check if they have the same dimension.

In [5]:
# try your own words and check if they have the same dimension of the cat vector
############# YOUR CODE HERE ################
computer = nlp.vocab['computer']

print(len(computer.vector))

300


After retrieving the words and their vector representations, we can use the built-in similarity function (which implements cosine similarity) to calculate word similarity based on these vectors. Is 'cat' more similar to 'dog' than 'car'? Can you find some properties of cosine similarity?

In [6]:
# you can calculate the similarity between words using
# the built-in 'similarity' function
print('The similarity between cat and cat:', cat.similarity(cat))
print('The similarity between cat and dog:', cat.similarity(dog))
print('The similarity between dog and cat:', dog.similarity(cat))
print('The similarity between cat and car:', cat.similarity(car))
print('The similarity between dog and car:', dog.similarity(car))

The similarity between cat and cat: 1.0
The similarity between cat and dog: 0.8220816850662231
The similarity between dog and cat: 0.8220816850662231
The similarity between cat and car: 0.19698593020439148
The similarity between dog and car: 0.3250025510787964


Now let's try some other words. Also, try to calculate the cosine similarity between 'hotel' and 'motel' and the cosine similarity between 'hotel' and 'hospital'. Which one is more similar to 'hotel'? 'motel' or 'hospital'?

In [7]:
# calculate the similarity of your own words using the built-in function
############# YOUR CODE HERE ################
hotel = nlp.vocab['hotel']
motel = nlp.vocab['motel']
hospital = nlp.vocab['hospital']

# what is the similarity between (hotel, motel) and (hotel, hospital)
############# YOUR CODE HERE ################
print(hotel.similarity(motel))

print(hotel.similarity(hospital))


0.7972080111503601
0.4250935912132263


Let's compute the cosine similarity manually using its definition below. Then check if the result is the same as the one calculated by the built-in function. \\
<br>
$cosine\_similarity(A, B) = \frac{A \cdot B}{\left \| A \right \|\left \| B \right \|}$

In [8]:
# try to calculate cosine similarity manually
'''
cosine of V1 and V2 = dot product of V1 and V2 / product of V1 norm and V2 norm
To get the vector representation of a word, use .vector, e.g. car.vector
To calculate the dot product of two vectors V1 and V2, use np.dot(V1, V2)
To get the norm of a word vector, use .vector_norm, e.g. car.vector_norm,
alternatively you can use np.linalg.norm(V1) to calculate the norm of V1
'''
############# YOUR CODE HERE ################
cosine_dog_car = np.dot(dog.vector, car.vector)/(dog.vector_norm*car.vector_norm)
print('The similarity between dog and car calculated manually:', cosine_dog_car)
#############################################

The similarity between dog and car calculated manually: 0.32500255


Now we know how to compare the similarity of two words using pretrained Word2Vec model. We can also use it to help us find semantically similar words, that is given a word retrieve similar words from the vocabulary. \\
<br>
The Python spaCy library hasn't provided such a function to do precisely this yet. We could use other NLP and machine learning libraries, such as [gensim](https://radimrehurek.com/gensim/), to do this with a simple function call. But the implementation is not hard, so let's give it a try! In our customized function, we first find all the words in our vocabulary (that has vector representations). Then we calculate the cosine similarity between our query word and each word in the vocabulary. We sort the similarity score in descending order. Finally, we retrieve the top n most similar words.

In [9]:
# function to find similar words
def most_similar(word, topn=10):
    allwords = [w for w in nlp.vocab if w.has_vector and w.is_lower and w.lower_ != word.lower_]  # get all words in the vocabulary
    by_similarity = sorted(allwords, key=lambda w: word.similarity(w), reverse=True)  # sort words by similarity in descending order
    return by_similarity[:topn]

In [10]:
# find similar words
cat_similar = [w.text for w in most_similar(dog)]
print('Similar words to cat: ', cat_similar)

Similar words to cat:  ['cat', 'you', 'car', 'somethin’', 'lovin’', "'cause", 'lovin', 'she', "somethin'", 'it']


### Word Analogy
One interesting finding for the Word2Vec model is that it embeds some analogical relationships between words. \\
<br>
*Man is to Woman as King is to Queen* \\
Man - Woman = King - Queen \\
<br>
*Paris is to France as Madrid is to Spain* \\
Paris - France = Madrid - Spain \\
<br>
These relationships can be reconstructed using word embeddings. \\
<br>
![analogy](https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2017/06/06062705/Word-Vectors.png)

In [11]:
# word analogy example
# king is to man as what is to woman?
king = nlp.vocab['king']
man = nlp.vocab['man']
woman = nlp.vocab['woman']

# resulting vector
result = king.vector - man.vector + woman.vector

# function to compute cosine similarity
cosine = lambda v1, v2: np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [12]:
# what word does the 'result' vector closely correspond to?

# we can first check if the 'result' vector is similar to the 'queen' vector
############# YOUR CODE HERE ################
queen = nlp.vocab['queen']
print('Similarity between queen and result:', cosine(result, queen.vector))
#############################################

# find all words in our vocabulary (nlp.vocab),
# make sure to just retrieve lower case words
# and words that actually have vectors (.has_vector)
# and filter out 'king', 'man', and 'woman'
############# YOUR CODE HERE ################
allwords = [w for w in nlp.vocab if w.has_vector and w.is_lower and w.lower_ != 'king' and w.lower_ != 'man' and w.lower_ != 'woman']
#############################################

# calculate the cosine similarity between the 'result' vector
# and all word vectors in our vocabulary
# sort by similarity and print out the most similar one
############# YOUR CODE HERE ################
candidates = sorted(allwords, key=lambda w: cosine(result, w.vector), reverse=True)
print([c.text for c in candidates[:5]])

Similarity between queen and result: 0.6178015
['queen', 'and', 'that', 'r.', 'where']


Let's try: \\
Paris - France = Madrid - Spain

In [13]:
# another example
# Paris is to France as Madrid is to what?
############# YOUR CODE HERE ################
Paris = nlp.vocab['Paris']
France = nlp.vocab['France']
Madrid = nlp.vocab['Madrid']

maybe_Spain = France.vector - Paris.vector + Madrid.vector

allwords = [w for w in nlp.vocab if w.has_vector and w.lower_ != 'paris' and w.lower_ != 'madrid' and w.lower_ != 'france']
candidates = sorted(allwords, key=lambda w: cosine(maybe_Spain, w.vector), reverse=True)
print([c.text for c in candidates[:5]])
#############################################

['and', 'king', 'Colo', 'Calif', 'were']


### Sentence/Document Level Similarity
Using word embeddings, we can also calculate similarity between sentences and documents. More advanced models such as Doc2Vec or neural networks can be used, but in this tutorial we will continue to use Word2Vec model to calculate document similarity. Since sentences and documents are composed of words, one easy way to obtain vector representations for sentences/documents is to calculate the average vectors of words. \\
<br>
Let's try to calculate the similarity among these three sentences:


1.   Cats are beautiful animals.
2.   Some gorgeous creatures are felines.
3.   Dolphins are swimming mammals.



In [14]:
# Word2Vec model does not provide vector representations for sentences
# or documents. How is the similarity between sentences computed?
# Since sentences are composed of words, an easy way to obtain the vector
# representations of sentences is by averaging the vectors of each word in
# the sentence.
############# YOUR CODE HERE ################
s1 = (nlp.vocab['Cats'].vector + nlp.vocab['are'].vector + nlp.vocab['beautiful'].vector + \
    nlp.vocab['animals'].vector + nlp.vocab['.'].vector)/5
s2 = (nlp.vocab['Some'].vector + nlp.vocab['gorgeous'].vector + nlp.vocab['creatures'].vector + \
    nlp.vocab['are'].vector + nlp.vocab['felines'].vector + nlp.vocab['.'].vector)/6
s3 = (nlp.vocab['Dolphins'].vector + nlp.vocab['are'].vector + nlp.vocab['swimming'].vector + \
    nlp.vocab['mammals'].vector + nlp.vocab['.'].vector)/5

print(cosine(s1, s2))

print(cosine(s1, s3))



0.9067516
0.9037428


In [15]:
# spaCy also supports similarity calculation between sentences and documents
target = nlp("Cats are beautiful animals.")  # text about cats

doc1 = nlp("Some gorgeous creatures are felines.")  # text about cats
doc2 = nlp("Dolphins are swimming mammals.")  # text about dolphins

print('Similarity between target and doc1:', target.similarity(doc1))
print('Similarity between target and doc1:', target.similarity(doc2))

Similarity between target and doc1: 0.9067516716813709
Similarity between target and doc1: 0.9037427153904276


### Word Embeddings Visualization
Since the word vectors we use have 300 dimensions, we cannot visualize them. One natural way is to apply dimension reduction first and then visualize them. We use a popular dimension reduction technique called [t-SNE](https://lvdmaaten.github.io/tsne/) (you can also use PCA) to reduce the word vectors to 2D and then plot the words in our word analogy example to see if we can find some pattern visually. \\
<br>
An interactive visualization of word embeddings can be found here: \\
[https://projector.tensorflow.org/](https://projector.tensorflow.org/)

In [17]:
# use t-SNE to do dimension reduction, from 300d to 2d
tsne_model = TSNE(n_components=10)

# get transformed vectors
data = np.array([king.vector, man.vector, queen.vector, woman.vector])
data_2d = tsne_model.fit_transform(data)

labels = ['king', 'man', 'queen', 'woman']

# plot the 2d vectors and show their labels
plt.scatter(data_2d[:, 0], data_2d[:, 1], s=100)
for i, txt in enumerate(labels):
    plt.annotate(txt, (data_2d[i,0], data_2d[i,1]), xytext=(2, 3), textcoords='offset points')
plt.show()

ValueError: perplexity must be less than n_samples

### Sentiment Analysis
The major reason for coming up with word embedding models is that we want to use these embeddings which encode the word semantics to help us tackle problems related with natural language. \\
<br>
One such task is sentiment analysis. By analyzing the sentiment of texts, we want to understand whether a given sentence/document is positive or negative. For example, 'the weather is so nice today' has a positive sentiment whereas 'he is bored by the movie' has a negative sentiment. \\
<br>
In this tutorial, we want to use the word embeddings combined with a simple machine learning model ([logistic regression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)) to do sentiment analysis. Logistic regression is a linear classification model and in our case we want to classify whether a given sentence is positive or negative. So it's a binary classification. \\
<br>
![logistic](https://rasbt.github.io/mlxtend/user_guide/classifier/SoftmaxRegression_files/logistic_regression_schematic.png)
<br>
Our training data contains 2,748 from Yelp reviews, IMDB movie reviews, and Amazon reviews. In the dataset, 1 means positive and 0 means negative. The original data can be downloaded from [here](https://www.kaggle.com/rahulin05/sentiment-labelled-sentences-data-set/data), the combined file can be downloaded from [here](https://drive.google.com/file/d/1knrjvDNkiXtviXBoLm5OJY45_kcCmDxe/view?usp=sharing).

In [ ]:
# load files into the environment
from google.colab import files
uploaded = files.upload()

In [ ]:
# read data
data_raw = []
with open('combined_training.txt', newline='') as fr:
    reader = csv.reader(fr, delimiter='\t')
    for row in reader:
        data_raw.append([row[0], int(row[1])])

# print the number of data
print(len(data_raw))

# print the last data item
print(data_raw[-1])

In [ ]:
x_train = np.array([nlp(d[0]).vector for d in data_raw])
y_train = np.array([d[1] for d in data_raw])

# print the dimension of x
print(x_train.shape)

# print the dimension of y
print(y_train.shape)

# double check
print(nlp(data_raw[-1][0]).text, y_train[-1])

In [ ]:
logreg = linear_model.LogisticRegression()
logreg.fit(x_train, y_train)

In [ ]:
# predict using trained model
predict = logreg.predict(np.array([nlp('the weather today is pleasant').vector, nlp('the food in this restaurant is beyond my expectation').vector]))
print(predict)